# 02 - Cluster Merge
**The intent of this notebook is to show some of the working logic that went into how we figured out the processes that were engineered in the `02 - Cluster Merge Notebook`**

The initial intent of this notebook is to combine previously formed dataframes to better inform our upcoming EDA steps. Additionally, within this notebook, we are loading and preparing a file to use for measuring our clusters in the EDA stage.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime as dt

In [2]:
np.random.seed(42)

In [3]:
clusters = pd.read_csv('./development_source_data/2018_with_clusters.csv')
clusters = clusters.set_index('State')
clusters.head()

,Total Nonfarm,Manufacturing,"Trade, Transportation, and Utils",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Other Services,Government,...,pct_Financial Activities,pct_Professional & Business Services,pct_Education & Health Services,pct_Leisure & Hospitality,pct_Other Services,pct_Government,"pct_Mining, Logging and Construction",clusters-4,clusters-3,clusters-5
State,,,,,,,,,,,,,,,,,,,,,
Alabama,2046.258,266.992,383.467,21.092,96.258,244.242,245.167,205.933,97.117,386.792,...,0.0470,0.1194,0.1198,0.1006,0.0475,0.1890,0.0485,0,2,0
Alaska,327.658,12.500,64.408,5.608,11.767,27.358,50.442,35.650,11.058,80.375,...,0.0359,0.0835,0.1539,0.1088,0.0337,0.2453,0.0870,0,2,3
Arizona,2857.717,171.450,534.525,47.558,220.008,434.033,445.417,325.975,92.442,415.858,...,0.0770,0.1519,0.1559,0.1141,0.0323,0.1455,0.0596,2,0,0
Arkansas,1267.492,160.800,253.408,12.433,61.058,145.425,191.575,118.267,55.650,212.108,...,0.0482,0.1147,0.1511,0.0933,0.0439,0.1673,0.0448,0,2,3
California,17172.225,1323.550,3045.983,542.850,837.875,2670.217,2722.283,1993.142,571.667,2581.483,...,0.0488,0.1555,0.1585,0.1161,0.0333,0.1503,0.0514,3,1,4


In [4]:
# Looking at unique clusters
clusters['clusters-5'].unique()

array([0, 3, 4, 1, 2])

In [5]:
# Looking at which states make up a particular cluster
clusters[clusters['clusters-5'] == 0].index

Index(['Alabama', 'Arizona', 'Colorado', 'Indiana', 'Kentucky', 'Louisiana',
       'Maryland', 'Massachusetts', 'Minnesota', 'Missouri', 'Oregon',
       'South Carolina', 'Tennessee', 'Washington', 'Wisconsin'],
      dtype='object', name='State')

In [6]:
# Testing out a regex method for future function
re.findall('\d+', 'clusters-5')[0]

'5'

#### Function to return clusters from dataframe
Associating the states with the cluster according to the KMeans outputs

In [7]:
def get_clusters(df, column_list):
    
    all_clusters = {}
    
    for column in column_list:
        
        n_cluster = int(re.findall('\d+', column)[0])
        
        state_clusters = {}

        for cluster in df[column].unique():
            state_clusters[cluster] = list(df[df[column] == cluster].index)
    
        all_clusters[n_cluster] = state_clusters
    
    return all_clusters
cluster_labels = get_clusters(clusters, ['clusters-3', 'clusters-4', 'clusters-5'])
cluster_labels[4][1]

['Florida', 'New York', 'Texas']

##### The next few cells were pre-work to understanding how to get the next function to work

In [8]:
bama = pd.read_csv('./development_state_employment/Alabama_employment.csv')
bama.head()

,Year,Period,labor force participation rate,employment-population ratio,labor force,employment,unemployment,unemployment rate
0,2012,Jan,58.3,53.4,2179750.0,1993523.0,186227.0,8.5
1,2012,Feb,58.2,53.3,2175635.0,1992089.0,183546.0,8.4
2,2012,Mar,58.1,53.2,2173644.0,1991104.0,182540.0,8.4
3,2012,Apr,58.1,53.2,2173494.0,1991037.0,182457.0,8.4
4,2012,May,58.1,53.2,2174754.0,1992484.0,182270.0,8.4


In [9]:
# Creating a date column based on Year, and setting to index
bama['date'] = pd.to_datetime(bama['Year'].astype(str) + '-' + bama['Period'])
bama = bama.set_index('date')
bama.head()

,Year,Period,labor force participation rate,employment-population ratio,labor force,employment,unemployment,unemployment rate
date,,,,,,,,
2012-01-01,2012,Jan,58.3,53.4,2179750.0,1993523.0,186227.0,8.5
2012-02-01,2012,Feb,58.2,53.3,2175635.0,1992089.0,183546.0,8.4
2012-03-01,2012,Mar,58.1,53.2,2173644.0,1991104.0,182540.0,8.4
2012-04-01,2012,Apr,58.1,53.2,2173494.0,1991037.0,182457.0,8.4
2012-05-01,2012,May,58.1,53.2,2174754.0,1992484.0,182270.0,8.4


In [10]:
temp_frame = pd.DataFrame()
temp_frame = temp_frame.append(bama)
temp_frame

,Year,Period,labor force participation rate,employment-population ratio,labor force,employment,unemployment,unemployment rate
date,,,,,,,,
2012-01-01,2012,Jan,58.3,53.4,2179750.0,1993523.0,186227.0,8.5
2012-02-01,2012,Feb,58.2,53.3,2175635.0,1992089.0,183546.0,8.4
2012-03-01,2012,Mar,58.1,53.2,2173644.0,1991104.0,182540.0,8.4
2012-04-01,2012,Apr,58.1,53.2,2173494.0,1991037.0,182457.0,8.4
2012-05-01,2012,May,58.1,53.2,2174754.0,1992484.0,182270.0,8.4
...,...,...,...,...,...,...,...,...
2021-09-01,2021,Sep,56.3,54.5,2240353.0,2168364.0,71989.0,3.2
2021-10-01,2021,Oct,56.3,54.5,2241442.0,2170873.0,70569.0,3.1
2021-11-01,2021,Nov,56.2,54.5,2242078.0,2172390.0,69688.0,3.1


In [11]:
# Testing a for loop on dictionary
for key in cluster_labels.keys():
    print(key)

3
4
5


#### Function to append state column to Unemployment data
Grouped by Clusters

In [12]:
for rank in cluster_labels.keys():
    
    for cluster in cluster_labels[rank].keys():
        
        cluster_df = pd.DataFrame()

        for state in cluster_labels[rank][cluster]:
            state_df = pd.read_csv(f'./development_state_employment/{state}_employment.csv')
            state_df['date'] = pd.to_datetime(state_df['Year'].astype(str) + '-' + state_df['Period'])
            state_df.set_index('date', inplace=True)
            state_df['State'] = state

            cluster_df = cluster_df.append(state_df)

        file_path = './development_state_employment/'
        csv_name = f'KMeans-{rank}_Cluster-{cluster}_employment.csv'
        cluster_df.to_csv(f'{file_path}{csv_name}', index_label='date')

##### Read in Unemployment data with State column

In [13]:
pd.read_csv('./development_state_employment/KMeans-5_Cluster-0_employment.csv', index_col='date')

,Year,Period,labor force participation rate,employment-population ratio,labor force,employment,unemployment,unemployment rate,State
date,,,,,,,,,
2012-01-01,2012,Jan,58.3,53.4,2179750.0,1993523.0,186227.0,8.5,Alabama
2012-02-01,2012,Feb,58.2,53.3,2175635.0,1992089.0,183546.0,8.4,Alabama
2012-03-01,2012,Mar,58.1,53.2,2173644.0,1991104.0,182540.0,8.4,Alabama
2012-04-01,2012,Apr,58.1,53.2,2173494.0,1991037.0,182457.0,8.4,Alabama
2012-05-01,2012,May,58.1,53.2,2174754.0,1992484.0,182270.0,8.4,Alabama
...,...,...,...,...,...,...,...,...,...
2021-09-01,2021,Sep,66.5,64.3,3135470.0,3030203.0,105267.0,3.4,Wisconsin
2021-10-01,2021,Oct,66.4,64.3,3134799.0,3035515.0,99284.0,3.2,Wisconsin
2021-11-01,2021,Nov,66.4,64.3,3134692.0,3037765.0,96927.0,3.1,Wisconsin


#### Read in Quarterly Personal Income data
The following cells will transform the raw data into the usable format for modeling and EDA.
Please note that running some of these cells more than once will cause the cell to break.

In [14]:
personal_income = pd.read_csv('./development_state_metrics/quarterly_personal_income_raw.csv')
personal_income.drop(columns=['GeoFips', 'LineCode', 'Description'], inplace=True)
personal_income.head()

,GeoName,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,2014:Q1,...,2019:Q2,2019:Q3,2019:Q4,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3
0,Alabama,171805.2,173476.2,171972.3,176192.9,174319.6,175041.3,176099.1,175281.1,176964.1,...,215125.8,216690.0,218325.6,220503.8,239992.0,228757.7,225741.7,260165.8,237569.3,239070.3
1,Alabama,4811070.0,4814535.0,4819450.0,4824247.0,4826744.0,4829571.0,4833698.0,4837723.0,4840041.0,...,4905709.0,4910975.0,4916419.0,4919385.0,4920954.0,4924127.0,4928670.0,4930141.0,4932200.0,4936526.0
2,Alabama,35710.0,36032.0,35683.0,36522.0,36115.0,36244.0,36432.0,36232.0,36563.0,...,43852.0,44124.0,44407.0,44823.0,48769.0,46456.0,45802.0,52770.0,48167.0,48429.0
3,Alaska *,38971.6,39289.6,39137.2,39665.6,38716.5,38734.6,39155.8,39305.2,40486.2,...,45149.9,45450.0,45628.5,45416.0,48729.7,45631.7,45943.8,50908.5,48328.8,48619.0
4,Alaska *,727712.0,729758.0,731774.0,733633.0,735202.0,736761.0,737598.0,737598.0,737379.0,...,733948.0,733448.0,733072.0,732407.0,731605.0,730832.0,730138.0,729197.0,728469.0,727870.0


In [15]:
personal_income.dropna(inplace=True)

In [16]:
# Testing pattern in regex for next cell
re.findall('\w+', 'Alaska *')[0]

'Alaska'

In [17]:
# Removing asterisk from Hawaii and Alaska (Noting that data prior to 1950 are not available)
personal_income['GeoName'] = personal_income['GeoName'].apply(lambda s: re.findall('\w+\s*\w+\s*\w+', s)[0])

In [18]:
personal_income.set_index('GeoName', inplace=True)
personal_income.head()

,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,2014:Q1,2014:Q2,...,2019:Q2,2019:Q3,2019:Q4,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3
GeoName,,,,,,,,,,,,,,,,,,,,,
Alabama,171805.2,173476.2,171972.3,176192.9,174319.6,175041.3,176099.1,175281.1,176964.1,179958.8,...,215125.8,216690.0,218325.6,220503.8,239992.0,228757.7,225741.7,260165.8,237569.3,239070.3
Alabama,4811070.0,4814535.0,4819450.0,4824247.0,4826744.0,4829571.0,4833698.0,4837723.0,4840041.0,4842263.0,...,4905709.0,4910975.0,4916419.0,4919385.0,4920954.0,4924127.0,4928670.0,4930141.0,4932200.0,4936526.0
Alabama,35710.0,36032.0,35683.0,36522.0,36115.0,36244.0,36432.0,36232.0,36563.0,37164.0,...,43852.0,44124.0,44407.0,44823.0,48769.0,46456.0,45802.0,52770.0,48167.0,48429.0
Alaska,38971.6,39289.6,39137.2,39665.6,38716.5,38734.6,39155.8,39305.2,40486.2,41091.7,...,45149.9,45450.0,45628.5,45416.0,48729.7,45631.7,45943.8,50908.5,48328.8,48619.0
Alaska,727712.0,729758.0,731774.0,733633.0,735202.0,736761.0,737598.0,737598.0,737379.0,737141.0,...,733948.0,733448.0,733072.0,732407.0,731605.0,730832.0,730138.0,729197.0,728469.0,727870.0


In [19]:
quarters = pd.Series(personal_income.columns)
quarters

0     2012:Q1
1     2012:Q2
2     2012:Q3
3     2012:Q4
4     2013:Q1
5     2013:Q2
6     2013:Q3
7     2013:Q4
8     2014:Q1
9     2014:Q2
10    2014:Q3
11    2014:Q4
12    2015:Q1
13    2015:Q2
14    2015:Q3
15    2015:Q4
16    2016:Q1
17    2016:Q2
18    2016:Q3
19    2016:Q4
20    2017:Q1
21    2017:Q2
22    2017:Q3
23    2017:Q4
24    2018:Q1
25    2018:Q2
26    2018:Q3
27    2018:Q4
28    2019:Q1
29    2019:Q2
30    2019:Q3
31    2019:Q4
32    2020:Q1
33    2020:Q2
34    2020:Q3
35    2020:Q4
36    2021:Q1
37    2021:Q2
38    2021:Q3
dtype: object

In [20]:
quarters = quarters.apply(lambda s: re.sub(':','-', s))
quarters

0     2012-Q1
1     2012-Q2
2     2012-Q3
3     2012-Q4
4     2013-Q1
5     2013-Q2
6     2013-Q3
7     2013-Q4
8     2014-Q1
9     2014-Q2
10    2014-Q3
11    2014-Q4
12    2015-Q1
13    2015-Q2
14    2015-Q3
15    2015-Q4
16    2016-Q1
17    2016-Q2
18    2016-Q3
19    2016-Q4
20    2017-Q1
21    2017-Q2
22    2017-Q3
23    2017-Q4
24    2018-Q1
25    2018-Q2
26    2018-Q3
27    2018-Q4
28    2019-Q1
29    2019-Q2
30    2019-Q3
31    2019-Q4
32    2020-Q1
33    2020-Q2
34    2020-Q3
35    2020-Q4
36    2021-Q1
37    2021-Q2
38    2021-Q3
dtype: object

In [21]:
quarters = pd.to_datetime(quarters)
quarters

0    2012-01-01
1    2012-04-01
2    2012-07-01
3    2012-10-01
4    2013-01-01
5    2013-04-01
6    2013-07-01
7    2013-10-01
8    2014-01-01
9    2014-04-01
10   2014-07-01
11   2014-10-01
12   2015-01-01
13   2015-04-01
14   2015-07-01
15   2015-10-01
16   2016-01-01
17   2016-04-01
18   2016-07-01
19   2016-10-01
20   2017-01-01
21   2017-04-01
22   2017-07-01
23   2017-10-01
24   2018-01-01
25   2018-04-01
26   2018-07-01
27   2018-10-01
28   2019-01-01
29   2019-04-01
30   2019-07-01
31   2019-10-01
32   2020-01-01
33   2020-04-01
34   2020-07-01
35   2020-10-01
36   2021-01-01
37   2021-04-01
38   2021-07-01
dtype: datetime64[ns]

In [22]:
quarters.values

array(['2012-01-01T00:00:00.000000000', '2012-04-01T00:00:00.000000000',
       '2012-07-01T00:00:00.000000000', '2012-10-01T00:00:00.000000000',
       '2013-01-01T00:00:00.000000000', '2013-04-01T00:00:00.000000000',
       '2013-07-01T00:00:00.000000000', '2013-10-01T00:00:00.000000000',
       '2014-01-01T00:00:00.000000000', '2014-04-01T00:00:00.000000000',
       '2014-07-01T00:00:00.000000000', '2014-10-01T00:00:00.000000000',
       '2015-01-01T00:00:00.000000000', '2015-04-01T00:00:00.000000000',
       '2015-07-01T00:00:00.000000000', '2015-10-01T00:00:00.000000000',
       '2016-01-01T00:00:00.000000000', '2016-04-01T00:00:00.000000000',
       '2016-07-01T00:00:00.000000000', '2016-10-01T00:00:00.000000000',
       '2017-01-01T00:00:00.000000000', '2017-04-01T00:00:00.000000000',
       '2017-07-01T00:00:00.000000000', '2017-10-01T00:00:00.000000000',
       '2018-01-01T00:00:00.000000000', '2018-04-01T00:00:00.000000000',
       '2018-07-01T00:00:00.000000000', '2018-10-01

In [23]:
# Transpose to add datetime column
personal_income = personal_income.T

In [24]:
personal_income['Quarter'] = quarters.values

In [25]:
quarters

0    2012-01-01
1    2012-04-01
2    2012-07-01
3    2012-10-01
4    2013-01-01
5    2013-04-01
6    2013-07-01
7    2013-10-01
8    2014-01-01
9    2014-04-01
10   2014-07-01
11   2014-10-01
12   2015-01-01
13   2015-04-01
14   2015-07-01
15   2015-10-01
16   2016-01-01
17   2016-04-01
18   2016-07-01
19   2016-10-01
20   2017-01-01
21   2017-04-01
22   2017-07-01
23   2017-10-01
24   2018-01-01
25   2018-04-01
26   2018-07-01
27   2018-10-01
28   2019-01-01
29   2019-04-01
30   2019-07-01
31   2019-10-01
32   2020-01-01
33   2020-04-01
34   2020-07-01
35   2020-10-01
36   2021-01-01
37   2021-04-01
38   2021-07-01
dtype: datetime64[ns]

In [26]:
personal_income['Quarter']

2012:Q1   2012-01-01
2012:Q2   2012-04-01
2012:Q3   2012-07-01
2012:Q4   2012-10-01
2013:Q1   2013-01-01
2013:Q2   2013-04-01
2013:Q3   2013-07-01
2013:Q4   2013-10-01
2014:Q1   2014-01-01
2014:Q2   2014-04-01
2014:Q3   2014-07-01
2014:Q4   2014-10-01
2015:Q1   2015-01-01
2015:Q2   2015-04-01
2015:Q3   2015-07-01
2015:Q4   2015-10-01
2016:Q1   2016-01-01
2016:Q2   2016-04-01
2016:Q3   2016-07-01
2016:Q4   2016-10-01
2017:Q1   2017-01-01
2017:Q2   2017-04-01
2017:Q3   2017-07-01
2017:Q4   2017-10-01
2018:Q1   2018-01-01
2018:Q2   2018-04-01
2018:Q3   2018-07-01
2018:Q4   2018-10-01
2019:Q1   2019-01-01
2019:Q2   2019-04-01
2019:Q3   2019-07-01
2019:Q4   2019-10-01
2020:Q1   2020-01-01
2020:Q2   2020-04-01
2020:Q3   2020-07-01
2020:Q4   2020-10-01
2021:Q1   2021-01-01
2021:Q2   2021-04-01
2021:Q3   2021-07-01
Name: Quarter, dtype: datetime64[ns]

In [27]:
personal_income = personal_income.set_index('Quarter')

In [28]:
personal_income.to_csv(f'./development_state_metrics/Quarterly_Personal_Income.csv', index_label='Quarter')

In [29]:
# It works!
pd.read_csv('./development_state_metrics/quarterly_personal_income.csv', index_col='Quarter')

,Alabama,Alabama.1,Alabama.2,Alaska,Alaska.1,Alaska.2,Arizona,Arizona.1,Arizona.2,Arkansas,...,Washington.2,West Virginia,West Virginia.1,West Virginia.2,Wisconsin,Wisconsin.1,Wisconsin.2,Wyoming,Wyoming.1,Wyoming.2
Quarter,,,,,,,,,,,,,,,,,,,,,
2012-01-01,171805.2,4811070.0,35710.0,38971.6,727712.0,53554.0,234167.8,6526567.0,35879.0,106004.7,...,46478.0,65490.9,1857328.0,35261.0,243324.5,5715700.0,42571.0,30517.8,573272.0,53234.0
2012-04-01,173476.2,4814535.0,36032.0,39289.6,729758.0,53839.0,237047.1,6546070.0,36212.0,107754.5,...,47140.0,66139.4,1857306.0,35610.0,245255.9,5718783.0,42886.0,31226.6,575512.0,54259.0
2012-07-01,171972.3,4819450.0,35683.0,39137.2,731774.0,53483.0,235982.2,6567653.0,35931.0,106758.9,...,47098.0,65552.5,1857480.0,35291.0,244138.5,5724075.0,42651.0,30746.5,577517.0,53239.0
2012-10-01,176192.9,4824247.0,36522.0,39665.6,733633.0,54067.0,244039.0,6588918.0,37038.0,110991.0,...,48683.0,66411.3,1857212.0,35759.0,252250.1,5729554.0,44026.0,32307.7,579111.0,55788.0
2013-01-01,174319.6,4826744.0,36115.0,38716.5,735202.0,52661.0,240161.4,6606778.0,36351.0,107782.3,...,47353.0,65517.2,1855941.0,35301.0,245126.1,5732112.0,42764.0,30923.8,580443.0,53276.0
2013-04-01,175041.3,4829571.0,36244.0,38734.6,736761.0,52574.0,243169.5,6624834.0,36706.0,108226.1,...,47652.0,65636.9,1854979.0,35384.0,247482.6,5735486.0,43149.0,31067.0,581852.0,53393.0
2013-07-01,176099.1,4833698.0,36432.0,39155.8,737598.0,53086.0,244888.4,6647932.0,36837.0,108785.2,...,48174.0,65479.4,1854470.0,35309.0,248871.3,5740541.0,43353.0,31258.0,582749.0,53639.0
2013-10-01,175281.1,4837723.0,36232.0,39305.2,737598.0,53288.0,246790.0,6674067.0,36977.0,108677.5,...,48354.0,65657.6,1853790.0,35418.0,249909.4,5745383.0,43497.0,31711.5,582992.0,54394.0
2014-01-01,176964.1,4840041.0,36563.0,40486.2,737379.0,54905.0,252123.8,6697498.0,37644.0,111803.6,...,49671.0,66405.1,1852355.0,35849.0,252761.5,5748087.0,43973.0,32605.0,583010.0,55925.0
